In [34]:
from pytz import timezone
import pytz
from datetime import datetime

In [58]:
# retourne un objet de type tzinfo
paris = timezone('Europe/Paris')
eastern = timezone('US/Eastern')

In [47]:
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
now = datetime.now()
utc_now = datetime.utcnow()
before_0200 = datetime(2016, 12, 1, 6)

In [48]:
# timezone ignorant, le temps est celui de la machine
# il n'y a pas d'info sur le TZ dans cet objet
print(now.strftime(fmt), '(heure locale)')

# datetime actuel à UTC+0000
print(utc_now.strftime(fmt), '(heure UTC+0000)')

# ajoute la timezone paris à l'objet datetime passé à localize
paris_dt = paris.localize(now)
print(paris_dt.strftime(fmt), '(heure avec info sur la timezone)')

# on set la timezone paris à une date qui était en UTC+01:00 à l'époque (maintenant UTC+02:00)
paris_before_0200 = paris.localize(before_0200)
print(paris_before_0200.strftime(fmt), '(ça reconnait bien la tz de l\'époque)')

# utiliser tzinfo= ne marche pas !!!!
print(now.replace(tzinfo=paris).strftime(fmt), '(erreur)')

# converti un datetime contenant déjà une TZ à l'heure UTC+0000
# astimezone ne s'utilise que sur un objet ayant une timezone
print(paris_dt.astimezone(pytz.utc).strftime(fmt), '(timezone mise à UTC+0000)')

2017-04-06 11:13:06  (heure locale)
2017-04-06 09:13:06  (heure UTC+0000)
2017-04-06 11:13:06 CEST+0200 (heure avec info sur la timezone)
2016-12-01 06:00:00 CET+0100 (ça reconnait bien la tz de l'époque)
2017-04-06 11:13:06 LMT+0009 (erreur)
2017-04-06 09:13:06 UTC+0000 (timezone mise à UTC+0000)


In [49]:
paris_dt = paris.localize(now, is_dst=True)
print(paris_dt.strftime(fmt), '(is_dst = True)')

paris_dt = paris.localize(now, is_dst=False)
print(paris_dt.strftime(fmt), '(is_dst = False)')

paris_dt = paris.localize(now, is_dst=None)
print(paris_dt.strftime(fmt), '(is_dst = None)')

2017-04-06 11:13:06 CEST+0200 (is_dst = True)
2017-04-06 11:13:06 CEST+0200 (is_dst = False)
2017-04-06 11:13:06 CEST+0200 (is_dst = None)


In [117]:
#################################
# SOLUTION !!!!!!
#################################

fmt = '%Y-%m-%d %H:%M:%S %Z%z'
paris = timezone('Europe/Paris')

# Comment parser les datetime
dt = paris.localize(datetime(2016, 12, 1, 6))
print(dt.strftime(fmt))

# le timestamp et la timezone à envoyer à craft
print(dt.timestamp(), dt.strftime('%z'))

# ce qu'on récupère comme timestamp dans la liste de contextes
sample_time = Time(int(dt.timestamp()), timezone=paris)
print(sample_time.to_dict())

# ou si l'on veut l'objet datetime python depuis le timestamp
# si on ne fait pas ça, on a un object datetime sans timezone...
dt_from_ts = paris.localize(datetime.fromtimestamp(dt.timestamp()))
print(dt_from_ts.strftime(fmt))

# le timestamp à envoyer pour récupérer l'arbre
tree_dt = dt.replace(hour=0, minute=0, second=0)
print(tree_dt.timestamp(), tree_dt.strftime(fmt))

2016-12-01 06:00:00 CET+0100
1480568400.0 +0100
{'time_of_day': 6.0, 'day_of_week': 3, 'timestamp': 1480568400, 'timezone': '+01:00', 'utc_iso': '2016-12-01T06:00:00+01:00', 'month_of_year': 12, 'day_of_month': 1}
2016-12-01 06:00:00 CET+0100
1480546800.0 2016-12-01 00:00:00 CET+0100


In [141]:
paris.localize(datetime.now()).strftime(fmt)
datetime.now().date() > datetime(2016, 1, 12).date()

True

In [136]:
import pandas as pd
from datetime import datetime
import pytz

paris = pytz.timezone('Europe/Paris')
data = pd.DataFrame()
row = pd.Series()
dt = paris.localize(datetime(2016, 12, 1, 6))
row['datetime'] = dt

# la timezone est bien affichée
print(row['datetime'].strftime("%z"))

data = data.append(row, ignore_index=True)

# Mais une fois ajoutée au DataFrame, on a perdu l'information
print(type(data.iloc[0]['datetime']))
print(type(data.iloc[0]['datetime'].to_pydatetime()))
print(data.iloc[0]['datetime'].to_pydatetime().strftime('%z'))

# Dans certains cas il peut y avoir une erreur quand on ajoute une colonne 
# de datetime timezone-aware à un dataframe vide
# https://github.com/pandas-dev/pandas/issues/12985

+0100
<class 'pandas.tslib.Timestamp'>
<class 'datetime.datetime'>

